# Projeto 3 - Ciência dos Dados

#  Projeto UFC

Nome: Victor Vergara Arcoverde de Albuquerque Cavalcanti

Nome: Edgard Neto

Nome: Gabriel Yamashita

Nome: Henrique Mualem Marti



  ___
## Objetivo:

### O objetivo desse projeto é fazer um machine learning para poder prever qual lutador ganhará uma luta do UFC baseado no seu histórico. Para isso serão usados os dados de todas as lutas do UFC (mais de 5 mil) a fim de descobrir quais os fatores dos lutadores que impactam mais no resultado das lutas. 

[Database utilizado](https://www.kaggle.com/rajeevw/ufcdata#data.csv)

 ___
## Preparando o ambiente no jupyter:

### Imports:

In [9]:
import math
import os.path
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import random

import statsmodels.api as sm

import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

### Trabalhando com os Excels:

In [10]:
data = pd.read_excel("data.xlsx")
preproc = pd.read_excel("preprocessed_data.xlsx")
fighter = pd.read_excel("raw_fighter_details.xlsx")
total = pd.read_excel("raw_total_fight_data.xlsx")

data.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,...,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,...,0,2,0,5,Southpaw,165.10,167.64,125.0,32.0,31.0


In [11]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

In [12]:
data.Winner.value_counts()

Red     3470
Blue    1591
Draw      83
Name: Winner, dtype: int64

In [13]:
data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data_heavy['title_bout'] = data_heavy['title_bout'].map(bool_to_number)
data_heavy['Winner'] = data_heavy['Winner'].map(string_to_number)

C:\Users\Maulem\Spyder\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Maulem\Spyder\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


#### Blue = 0
#### Red = 1

In [14]:
data_heavy.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019,"Chicago, Illinois, USA",0,0,Heavyweight,3,0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,Jerin Valel,2019,"Ottawa, Ontario, Canada",1,0,Heavyweight,3,0,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0


In [15]:
data_heavy_util = data_heavy.drop(['Referee','date','location'], axis=1)
#dados que não se relacionam com os lutadores ou seus resultado

In [16]:
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,1,0,Heavyweight,3,0,0,0,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.0,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.0,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.0,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0


In [17]:
data_heavy_util.dropna(inplace=True)
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.00,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.00,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.00,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.00,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0
75,Alistair Overeem,Aleksei Oleinik,1,0,Heavyweight,5,0,2,0,4.25,...,6,0,0,9,Orthodox,193.04,203.20,265.0,41.0,38.0


#### Tirando os dados categóricos:

In [18]:
categoricas = [
    'R_fighter', 
    'B_fighter', 
    'weight_class', 
    'R_Stance', 
    'B_Stance', 
]

data_heavy_cat = data_heavy_util[categoricas].astype('category')
data_heavy_num = data_heavy_util.drop(categoricas, axis=1).astype('float')

In [19]:
X = data_heavy_num.drop('Winner', axis=1)
y = data_heavy_num['Winner']

### Separando os dados em testes e treinamento

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [21]:

model = RandomForestClassifier(n_estimators=10000)

model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [22]:
#Fatores que tem o maior peso na decisão da vitória
j=1
lista_j=list()
for i, f in sorted(list(zip(model.feature_importances_, X_train.columns)), reverse=True):
    print( j,'°',f'{f}: {i}')
    a=str(j)+'°'
    lista_j.append(a)
    j+=1


1 ° B_avg_BODY_att: 0.019286640156694814
2 ° B_avg_BODY_landed: 0.01765518797038849
3 ° B_avg_TOTAL_STR_att: 0.014683964471462283
4 ° B_avg_HEAD_att: 0.013998742695714975
5 ° B_avg_SIG_STR_att: 0.013984183928218446
6 ° B_Reach_cms: 0.013381457495274711
7 ° B_avg_opp_TOTAL_STR_landed: 0.012005493946471183
8 ° B_avg_SIG_STR_landed: 0.011581260610268564
9 ° B_avg_opp_TD_att: 0.01145416262859449
10 ° B_avg_DISTANCE_att: 0.011383110914048434
11 ° B_avg_opp_TOTAL_STR_att: 0.010907425502669528
12 ° R_avg_HEAD_landed: 0.010897752768734635
13 ° R_total_rounds_fought: 0.010744064765782307
14 ° R_avg_opp_SIG_STR_pct: 0.010708690935894494
15 ° R_avg_opp_CLINCH_att: 0.010693616193894395
16 ° R_age: 0.010690464431031551
17 ° R_avg_opp_GROUND_att: 0.010650931577344909
18 ° R_avg_opp_CLINCH_landed: 0.010629366199612016
19 ° R_avg_opp_GROUND_landed: 0.01038862540513557
20 ° R_avg_SIG_STR_landed: 0.010295298032364408
21 ° B_avg_HEAD_landed: 0.010209495744429073
22 ° R_avg_opp_TOTAL_STR_landed: 0.0100731

#### Criando um dataframe com os dados acima:

In [23]:
data={'Fator':X_train.columns ,'Correlação':model.feature_importances_,}
Fator_por_corr=pd.DataFrame(data)
Fator_por_corr=Fator_por_corr.sort_values(by='Correlação', ascending=False)
Fator_por_corr['Grau de Importância']=lista_j
Fator_por_corr = Fator_por_corr.set_index('Grau de Importância')
Fator_por_corr.head()

,Fator,Correlação
Grau de Importância,,
1°,B_avg_BODY_att,0.019287
2°,B_avg_BODY_landed,0.017655
3°,B_avg_TOTAL_STR_att,0.014684
4°,B_avg_HEAD_att,0.013999
5°,B_avg_SIG_STR_att,0.013984


___
## Verificando a performance


In [24]:
X_train.columns

Index(['title_bout', 'no_of_rounds', 'B_current_lose_streak',
       'B_current_win_streak', 'B_draw', 'B_avg_BODY_att', 'B_avg_BODY_landed',
       'B_avg_CLINCH_att', 'B_avg_CLINCH_landed', 'B_avg_DISTANCE_att',
       ...
       'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Height_cms', 'R_Reach_cms',
       'R_Weight_lbs', 'B_age', 'R_age'],
      dtype='object', length=136)

In [25]:
model.feature_importances_

array([0.00051677, 0.00083043, 0.0031684 , 0.00429304, 0.        ,
       0.01928664, 0.01765519, 0.00945423, 0.00884915, 0.01138311,
       0.00910839, 0.00785845, 0.00763198, 0.01399874, 0.0102095 ,
       0.00527072, 0.00971744, 0.00894293, 0.0070235 , 0.00864268,
       0.01398418, 0.01158126, 0.00802781, 0.00491581, 0.00811764,
       0.00555663, 0.00577309, 0.01468396, 0.00956127, 0.0041858 ,
       0.00438149, 0.00903811, 0.0094996 , 0.00869323, 0.00802824,
       0.00716814, 0.00769242, 0.00856039, 0.00876595, 0.01007013,
       0.00831222, 0.00496374, 0.00819314, 0.00694477, 0.00613653,
       0.00586548, 0.0096149 , 0.0078511 , 0.00807377, 0.00544421,
       0.01145416, 0.00707261, 0.00735306, 0.01090743, 0.01200549,
       0.00623547, 0.00885566, 0.00151244, 0.00040113, 0.00119506,
       0.00167396, 0.00323883, 0.00167203, 0.00076008, 0.00412928,
       0.00902287, 0.01338146, 0.00697413, 0.00498104, 0.00535321,
       0.        , 0.00840526, 0.00836058, 0.0077572 , 0.00780

In [26]:
y_pred = model.predict(X_test)

In [27]:
print(accuracy_score(y_test, y_pred))

0.6666666666666666


In [28]:
y_pred

array([1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.])

In [29]:
y_test.value_counts(True)

1.0    0.681818
0.0    0.303030
2.0    0.015152
Name: Winner, dtype: float64

In [30]:
data_heavy_num.shape

(264, 137)

### Conclusão desse teste:

Tendo um acurácia de 0.6667 não é um bom resultado, visto que o modelo praticamente sempre tem como resultado o vermelho como vencedor, e como a probabilidade do lutador vermelho ganhar é de 0.6212112 ele praticamente só acerta os que o vencedor é o vermelho e erra os que o azul é o vencedor.

Assim é possível concluir que é necessário desconsiderar algumas variáveis para tentar melhorar a acurácia.

___
## Montando a Regressão Logística


## !Ainda é necessário tirar os fatores que foram julgados ter um baixo grau de importância pelo teste anterior!

In [31]:
def preparo(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp,missing='drop')
    results = model.fit()
    return results

In [32]:
Y = data_heavy_num["Winner"]
data_heavy_sem_Winner=data_heavy_num.drop('Winner',axis=1) 
X=data_heavy_sem_Winner
#np.asarray(X)

In [33]:
result = preparo(X,Y)
result.summary()

C:\Users\Maulem\Spyder\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Winner   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     1.305
Date:                Wed, 13 Nov 2019   Prob (F-statistic):             0.0635
Time:                        22:56:22   Log-Likelihood:                -100.66
No. Observations:                 264   AIC:                             439.3
Df Residuals:                     145   BIC:                             864.9
Df Model:                         118                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            4.6535      2.927      1.590      0.114      -1.131      10.438
title_bout                      -0.0643      0.192     -0.334      0.739      -0.445       0.316
no_of_rounds                     0.0958      0.062      1.533      0.127      -0.028       0.219
B_current_lose_streak           -0.2204      0.073     -3.002      0.003      -0.366      -0.075
B_current_win_streak            -0.0361      0.037     -0.983      0.327      -0.109       0.037
B_draw                        1.562e-13   4.62e-14      3.377      0.001    6.48e-14    2.48e-13
B_avg_BODY_att                  -0.0701      0.040     -1.752      0.082      -0.149       0.009
B_avg_BODY_landed                0.0788      0.049      1.623      0.107      -0.017       0.175
B_avg_CLINCH_att                 0.0259      0.025      1.025      0.307      -0.024       0.076
B_avg_CLINCH_landed             -0.0234      0.035     -0.669      0.504      -0.092       0.046
B_avg_DISTANCE_att              -0.0151      0.014     -1.059      0.291      -0.043       0.013
B_avg_DISTANCE_landed            0.0141      0.019      0.727      0.469      -0.024       0.053
B_avg_GROUND_att                -0.0245      0.023     -1.059      0.291      -0.070       0.021
B_avg_GROUND_landed              0.0242      0.032      0.764      0.446      -0.038       0.087
B_avg_HEAD_att                  -0.0283      0.022     -1.283      0.201      -0.072       0.015
B_avg_HEAD_landed                0.0352      0.028      1.266      0.208      -0.020       0.090
B_avg_KD                         0.3686      0.193      1.906      0.059      -0.014       0.751
B_avg_LEG_att                    0.0846      0.059      1.436      0.153      -0.032       0.201
B_avg_LEG_landed                -0.0990      0.071     -1.386      0.168      -0.240       0.042
B_avg_PASS                       0.1028      0.075      1.375      0.171      -0.045       0.251
B_avg_REV                        0.7377      0.295      2.497      0.014       0.154       1.322
B_avg_SIG_STR_att               -0.0137      0.023     -0.606      0.546      -0.059       0.031
B_avg_SIG_STR_landed             0.0150      0.027      0.551      0.583      -0.039       0.069
B_avg_SIG_STR_pct               -1.5196      0.575     -2.644      0.009      -2.656      -0.384
B_avg_SUB_ATT                   -0.1195      0.137     -0.871      0.385      -0.391       0.152
B_avg_TD_att                    -0.0131      0.048     -0.274      0.784      -0.107       0.081
B_avg_TD_landed                 -0.0729      0.104     -0.701      0.484      -0.278       0.133
B_avg_TD_pct                     0.3372      0.275      1.228      0.222      -0.206       0.880
B_avg_TOTAL_STR_att              0.0405      0.026      1.568      0.119    

In [34]:
'''data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]
data_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)
data_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]
data_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)'''

"data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]\ndata_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)\ndata_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]\ndata_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)"

In [35]:
'''blue_winners_correlation = data_heavy_blue_winners.corr()
blue_winners_correlation.head()'''

'blue_winners_correlation = data_heavy_blue_winners.corr()\nblue_winners_correlation.head()'

In [36]:
'''red_winners_correlation = data_heavy_red_winners.corr()
red_winners_correlation.head()'''

'red_winners_correlation = data_heavy_red_winners.corr()\nred_winners_correlation.head()'

___
## Concluindo

   ___
## Aperfeiçoamento:


   ___
## Referências

[Como funciona regressão linear](https://www.saedsayad.com/logistic_regression.htm)

[Como usar a biblioteca Scikit-lear](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

[Como usar a Regressão Linear](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)